In [19]:
import cv2
import mediapipe as mp
import numpy as np
import time
import random
import sys
import pickle
import os

# ==========================================
# 1. MATEMÁTICAS (EL CEREBRO)
# ==========================================

def get_hand_features(landmarks):
    coords = np.array([[lm.x, lm.y] for lm in landmarks.landmark])
    centroid = np.mean(coords, axis=0)
    centered = coords - centroid
    max_dist = np.max(np.abs(centered))
    if max_dist > 0:
        normalized = centered / max_dist
    else:
        normalized = centered
    return normalized.flatten()

def get_face_features(landmarks):
    # IMPORTANTE: Usamos TODOS los puntos (956 datos) para compatibilidad
    coords = np.array([[lm.x, lm.y] for lm in landmarks.landmark])
    centroid = np.mean(coords, axis=0)
    centered = coords - centroid
    max_dist = np.max(np.abs(centered))
    if max_dist > 0:
        normalized = centered / max_dist
    else:
        normalized = centered
    return normalized.flatten()

# ==========================================
# 2. CONFIGURACIÓN
# ==========================================

FILE_MANOS = "../train/modelo_body_red_augmentation.pkl"
FILE_CARA = "../train/modelo_facial_red_augmentation.pkl"

NOMBRES_BONITOS = {
    "1_Mano_Der_Arriba": "Mano Arriba",
    "2_Mano_Izq_Arriba": "Mano Arriba",
    "3_Punos_Cerrados": "Puño",
    "4_Pulgar_Arriba": "Pulgar Arriba",
    "5_Victoria": "Victoria",
    "6_Rock": "Rock",
    "7_Llamada": "Llamada",
    "8_Ok": "OK",
    "1_Ojos_Cerrados": "Ojos Cerrados",
    "2_Cabeza_Der": "Cabeza Derecha",
    "3_Cabeza_Izq": "Cabeza Izquierda",
    "0_Neutro": "Neutro"
}

# ==========================================
# 3. JUEGO SIMON HARDCORE
# ==========================================

class SimonGame:
    def __init__(self):
        print("--- SIMON SAYS: HARDCORE MEMORY ---")
        
        # Cargar Modelos
        if not os.path.exists(FILE_MANOS) or not os.path.exists(FILE_CARA):
            raise FileNotFoundError("Faltan modelos .pkl")

        with open(FILE_MANOS, 'rb') as f: self.model_hand = pickle.load(f)
        with open(FILE_CARA, 'rb') as f: self.model_face = pickle.load(f)

        self.map_hand = {
            0: "0_Neutro", 1: "1_Mano_Der_Arriba", 2: "2_Mano_Izq_Arriba", 3: "3_Punos_Cerrados",
            4: "4_Pulgar_Arriba", 5: "5_Victoria", 6: "6_Rock", 7: "7_Llamada", 8: "8_Ok"
        }
        self.map_face = {
            0: "0_Neutro", 1: "1_Ojos_Cerrados", 2: "2_Cabeza_Der", 3: "3_Cabeza_Izq"
        }

        # MediaPipe
        self.mp_hands = mp.solutions.hands
        self.hands = self.mp_hands.Hands(max_num_hands=1, min_detection_confidence=0.5)
        self.mp_face = mp.solutions.face_mesh
        self.face_mesh = self.mp_face.FaceMesh(max_num_faces=1, refine_landmarks=True, min_detection_confidence=0.5)
        self.mp_draw = mp.solutions.drawing_utils

        # Lista de Acciones
        self.acciones_disponibles = []
        all_labels = list(self.map_hand.values()) + list(self.map_face.values())
        for label in all_labels:
            nombre_limpio = self.limpiar_nombre(label)
            if "Neutro" not in nombre_limpio and nombre_limpio not in self.acciones_disponibles:
                self.acciones_disponibles.append(nombre_limpio)

        # Variables de Estado
        self.cap = cv2.VideoCapture(0)
        self.state = "MENU"
        self.sequence = []          # Lista acumulativa de pasos
        self.current_step_idx = 0   # Por cuál paso vamos en la repetición actual
        self.score = 0
        self.timer_start = 0
        self.turn_time = 5.0        # Tiempo para hacer CADA gesto
        self.fail_reason = ""
        
        # Variables de detección
        self.last_gesture = "Neutro"
        self.consecutive_frames = 0
        self.waiting_neutral = False 

    def limpiar_nombre(self, raw_name):
        return NOMBRES_BONITOS.get(raw_name, raw_name)

    def detectar_gesto_actual(self, frame):
        """Devuelve el gesto detectado y la confianza."""
        rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        gesto = "Neutro"
        max_prob = 0.0

        # Mano
        res_hand = self.hands.process(rgb)
        if res_hand.multi_hand_landmarks:
            lm = res_hand.multi_hand_landmarks[0]
            self.mp_draw.draw_landmarks(frame, lm, self.mp_hands.HAND_CONNECTIONS)
            features = get_hand_features(lm)
            try:
                pred = self.model_hand.predict([features])[0]
                prob = np.max(self.model_hand.predict_proba([features]))
                if prob > 0.5:
                    label = self.limpiar_nombre(self.map_hand[pred])
                    if label != "Neutro":
                        gesto = label
                        max_prob = prob
            except: pass

        # Cara (Si mano < 0.8)
        if max_prob < 0.8:
            res_face = self.face_mesh.process(rgb)
            if res_face.multi_face_landmarks:
                lm = res_face.multi_face_landmarks[0]
                features = get_face_features(lm)
                try:
                    pred = self.model_face.predict([features])[0]
                    prob = np.max(self.model_face.predict_proba([features]))
                    if prob > 0.6:
                        label = self.limpiar_nombre(self.map_face[pred])
                        if label != "Neutro" and prob > max_prob:
                            gesto = label
                            max_prob = prob
                except: pass
        
        return gesto, max_prob

    def draw_ui(self, frame, main, sub="", color=(255, 255, 255)):
        H, W, _ = frame.shape
        overlay = frame.copy()
        cv2.rectangle(overlay, (0, 0), (W, 110), (0, 0, 0), -1)
        cv2.addWeighted(overlay, 0.6, frame, 0.4, 0, frame)
        cv2.putText(frame, main, (20, 50), cv2.FONT_HERSHEY_SIMPLEX, 1.0, color, 3)
        cv2.putText(frame, sub, (20, 90), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (200, 200, 200), 2)
        cv2.putText(frame, f"Score: {self.score}", (W-180, 50), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 255), 2)

    def add_sequence_step(self):
        """Añade un nuevo paso al final de la lista."""
        accion = random.choice(self.acciones_disponibles)
        # 30% prob de ser trampa
        es_trampa = random.random() < 0.3
        
        texto = f"Simon: {accion}"
        if es_trampa:
            texto = f"Modesto: {accion}"
            
        self.sequence.append({
            'action': accion,
            'text': texto,
            'is_trap': es_trampa
        })

    def run(self):
        print("--- JUEGO LISTO ---")
        try:
            while self.cap.isOpened():
                ret, frame = self.cap.read()
                if not ret: break
                frame = cv2.flip(frame, 1)
                H, W, _ = frame.shape
                
                # 1. Detección continua
                raw_gesto, prob = self.detectar_gesto_actual(frame)
                
                if raw_gesto == self.last_gesture:
                    self.consecutive_frames += 1
                else:
                    self.last_gesture = raw_gesto
                    self.consecutive_frames = 0
                
                gesto_estable = raw_gesto if self.consecutive_frames > 2 else "..."

                # Debug
                cv2.putText(frame, f"Detectado: {gesto_estable}", (20, H-30), 
                            cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)

                # --- MAQUINA DE ESTADOS ---

                if self.state == "MENU":
                    self.draw_ui(frame, "SIMON SAYS: MEMORY", "ESPACIO para empezar")
                    if cv2.waitKey(1) & 0xFF == 32:
                        self.sequence = []
                        self.score = 0
                        self.add_sequence_step() # Primer paso
                        self.state = "SHOW_NEW_STEP" # Vamos directo a enseñar SOLO el nuevo
                        self.current_step_idx = 0
                        self.timer_start = time.time()

                elif self.state == "SHOW_NEW_STEP":
                    # Solo mostramos el ÚLTIMO paso añadido
                    last_step = self.sequence[-1]
                    elapsed = time.time() - self.timer_start
                    
                    # CAMBIO: Color siempre blanco para no dar pistas
                    color = (255, 255, 255) 
                    
                    # Fondo negro grande en el centro para leer bien
                    cv2.rectangle(frame, (0, H//2 - 80), (W, H//2 + 80), (0,0,0), -1)
                    cv2.putText(frame, "NUEVO PASO:", (W//2 - 120, H//2 - 40), 
                                cv2.FONT_HERSHEY_SIMPLEX, 0.8, (200,200,200), 2)
                    cv2.putText(frame, last_step['text'], (50, H//2 + 20), 
                                cv2.FONT_HERSHEY_SIMPLEX, 1.5, color, 3)
                    
                    if elapsed > 3.0: # 3 segundos para memorizar
                        # Toca jugar desde el principio
                        self.state = "WAIT_NEUTRAL" 
                        self.current_step_idx = 0
                        self.waiting_neutral = True

                elif self.state == "WAIT_NEUTRAL":
                    # Pausa entre pasos hasta que el usuario se relaja
                    # NO MOSTRAMOS LA SIGUIENTE INSTRUCCIÓN AQUÍ para no dar pistas antes de tiempo
                    
                    self.draw_ui(frame, "¡VUELVE A NEUTRO!", "Preparando siguiente...", (255, 255, 0))
                    
                    # Si detectamos Neutro firme, pasamos al turno
                    if gesto_estable == "Neutro":
                        self.state = "PLAYER_TURN"
                        self.timer_start = time.time()

                elif self.state == "PLAYER_TURN":
                    step = self.sequence[self.current_step_idx]
                    elapsed = time.time() - self.timer_start
                    remaining = self.turn_time - elapsed
                    
                    idx_show = self.current_step_idx + 1
                    total = len(self.sequence)
                    
                    # EN PANTALLA: Solo mostramos "Repite paso X"
                    # El jugador tiene que recordar si era "Simon: Hola" o "Modesto: Hola"
                    instr = f"Repite el paso {idx_show}/{total}"
                    
                    self.draw_ui(frame, instr, f"Tiempo: {remaining:.1f}s")
                    
                    # LÓGICA DE VALIDACIÓN
                    round_passed = False
                    round_failed = False
                    
                    if step['is_trap']:
                        # --- MODESTO (Trampa) ---
                        # Usuario debe quedarse quieto. Si hace el gesto prohibido -> PIERDE.
                        if gesto_estable == step['action']:
                            round_failed = True
                            self.fail_reason = f"¡Era Modesto! No debias hacer {step['action']}"
                        elif remaining <= 0:
                            # Sobrevivió al tiempo
                            round_passed = True
                    else:
                        # --- SIMON (Correcto) ---
                        if gesto_estable == step['action']:
                            round_passed = True
                        elif remaining <= 0:
                            round_failed = True
                            self.fail_reason = f"¡Lento! Tenias que hacer {step['action']}"

                    # TRANSICIONES
                    if round_failed:
                        self.state = "GAMEOVER"
                        
                    elif round_passed:
                        # Feedback visual
                        cv2.circle(frame, (W-50, H-50), 40, (0, 255, 0), -1)
                        self.score += 10
                        
                        # Avanzar en la lista
                        self.current_step_idx += 1
                        
                        # ¿Quedan más pasos en la lista?
                        if self.current_step_idx < len(self.sequence):
                            # SÍ -> Volver a pedir Neutro antes del siguiente
                            self.state = "WAIT_NEUTRAL"
                        else:
                            # NO -> Secuencia completada
                            self.state = "SUCCESS_SEQUENCE"
                            self.timer_start = time.time()

                elif self.state == "SUCCESS_SEQUENCE":
                    cv2.rectangle(frame, (0, H//2 - 60), (W, H//2 + 60), (0,255,0), -1)
                    cv2.putText(frame, "¡RONDA COMPLETADA!", (W//2 - 250, H//2 + 10), cv2.FONT_HERSHEY_SIMPLEX, 1.2, (255,255,255), 3)
                    
                    if time.time() - self.timer_start > 2.0:
                        self.add_sequence_step() # Añadir uno más
                        self.state = "SHOW_NEW_STEP" # Mostrar SOLO el nuevo
                        self.timer_start = time.time()

                elif self.state == "GAMEOVER":
                    overlay = frame.copy()
                    cv2.rectangle(overlay, (0, 0), (W, H), (0, 0, 50), -1)
                    cv2.addWeighted(overlay, 0.7, frame, 0.3, 0, frame)
                    cv2.putText(frame, "GAME OVER", (W//2 - 180, H//2 - 50), cv2.FONT_HERSHEY_SIMPLEX, 2, (255, 255, 255), 4)
                    cv2.putText(frame, self.fail_reason, (50, H//2 + 20), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (200, 200, 255), 2)
                    cv2.putText(frame, "R: Reiniciar", (W//2 - 100, H - 100), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
                    
                    if cv2.waitKey(1) & 0xFF == ord('r'):
                        self.state = "MENU"

                cv2.imshow('SIMON MEMORY GAME', frame)
                if cv2.waitKey(1) & 0xFF == 27: break
        
        finally:
            self.cap.release()
            cv2.destroyAllWindows()

if __name__ == "__main__":
    game = SimonGame()
    game.run()

--- SIMON SAYS: HARDCORE MEMORY ---
--- JUEGO LISTO ---


c:\Users\lucia\anaconda33\envs\memo_env\lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '
c:\Users\lucia\anaconda33\envs\memo_env\lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '
c:\Users\lucia\anaconda33\envs\memo_env\lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '
c:\U